In [1]:
import sys
sys.path.append("/home/scorpio/ngsild-pythonclient-initial_ngsild_client/src/")

import ast
import json
from ngsildclient.client import Client
from ngsildclient.entity import Entity
import csv
import time
from datetime import datetime
import pandas as pd 
import numpy as np
from geojson.geometry import Point, Polygon, MultiLineString

client_obj = Client("localhost", "9090") # connect to scorpio

def publish(entity):
    response = client_obj.temporal_create(entity)
    #print(response)
    if response['status_code']> 400:
        print("ERROR: Cannot publish data from the NGSI client.")
    return response



In [2]:
num_rows_batch_publish=5000 # number of objects to be batch published to Scorpio

# Notes:
Consider every hexagon as an entity, and sensor data as properties

In [3]:
import pandas as pd
import geopandas as gpd

from geojson.geometry import Point, Polygon, MultiLineString


In [4]:
gdf_NO2 = pd.read_pickle('./../cgarrido/madrid_data/air_pollution/gdf_NO2.pkl')
gdf_O3 = pd.read_pickle('./../cgarrido/madrid_data/air_pollution/gdf_O3.pkl')
gdf_PM10 = pd.read_pickle('./../cgarrido/madrid_data/air_pollution/gdf_PM10.pkl')
gdf_PM2_5 = pd.read_pickle('./../cgarrido/madrid_data/air_pollution/gdf_PM2.5.pkl')
gdf_SO2 = pd.read_pickle('./../cgarrido/madrid_data/air_pollution/gdf_SO2.pkl')
gdf_CO = pd.read_pickle('./../cgarrido/madrid_data/air_pollution/gdf_CO.pkl')



In [5]:
gdf_NO2

,station_id,geometry,hexagon_id,hexagon_x,hexagon_y,altitude,datetime,year,month,day,hour,minute,weekday,data
0,8.0,POINT (-409913.520 4927391.691),0,0,0,672.0,2022-01-01 00:00:00,2022,1,1,0,0,5,35.0
1,38.0,POINT (-412675.857 4930900.318),5,1,0,699.0,2022-01-01 00:00:00,2022,1,1,0,0,5,56.0
2,60.0,POINT (-410738.954 4938949.357),48,2,-2,715.0,2022-01-01 00:00:00,2022,1,1,0,0,5,43.0
3,18.0,POINT (-415426.039 4923477.938),35,2,3,632.0,2022-01-01 00:00:00,2022,1,1,0,0,5,46.0
4,24.0,POINT (-417152.482 4927070.647),36,3,3,646.0,2022-01-01 00:00:00,2022,1,1,0,0,5,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770875,55.0,POINT (-398586.661 4933360.830),53,-3,-4,619.0,2022-12-31 23:45:00,2022,12,31,23,45,5,51.0
770876,40.0,POINT (-406486.304 4922508.219),12,-1,1,677.0,2022-12-31 23:45:00,2022,12,31,23,45,5,42.0
770877,39.0,POINT (-413166.342 4935683.000),30,2,-1,676.0,2022-12-31 23:45:00,2022,12,31,23,45,5,52.0
770878,48.0,POINT (-410810.432 4930073.380),5,1,0,680.0,2022-12-31 23:45:00,2022,12,31,23,45,5,52.0


In [6]:

gdf_list= [gdf_PM2_5,gdf_NO2, gdf_O3, gdf_PM10,  gdf_SO2, gdf_CO]
gdf_property_list=["PM2_5", "NO2", "O3", "PM10",  "SO2", "CO"]
entity_id_prefix =  "urn:ngsi-ld:AirQualityObserved:Madrid:"
entity_type = 'AirQualityObserved'


In [7]:
from pyproj import Transformer

transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326") # transform geometry to lat/lon
for g in gdf_list:
    poly = g['geometry']
    g['lat'] ,  g['lon'] =  transformer.transform(poly.centroid.x,poly.centroid.y) 

In [8]:
gdf_NO2

,station_id,geometry,hexagon_id,hexagon_x,hexagon_y,altitude,datetime,year,month,day,hour,minute,weekday,data,lat,lon
0,8.0,POINT (-409913.520 4927391.691),0,0,0,672.0,2022-01-01 00:00:00,2022,1,1,0,0,5,35.0,40.421553,-3.682316
1,38.0,POINT (-412675.857 4930900.318),5,1,0,699.0,2022-01-01 00:00:00,2022,1,1,0,0,5,56.0,40.445544,-3.707130
2,60.0,POINT (-410738.954 4938949.357),48,2,-2,715.0,2022-01-01 00:00:00,2022,1,1,0,0,5,43.0,40.500548,-3.689731
3,18.0,POINT (-415426.039 4923477.938),35,2,3,632.0,2022-01-01 00:00:00,2022,1,1,0,0,5,46.0,40.394782,-3.731836
4,24.0,POINT (-417152.482 4927070.647),36,3,3,646.0,2022-01-01 00:00:00,2022,1,1,0,0,5,37.0,40.419358,-3.747344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770875,55.0,POINT (-398586.661 4933360.830),53,-3,-4,619.0,2022-12-31 23:45:00,2022,12,31,23,45,5,51.0,40.462363,-3.580565
770876,40.0,POINT (-406486.304 4922508.219),12,-1,1,677.0,2022-12-31 23:45:00,2022,12,31,23,45,5,42.0,40.388148,-3.651529
770877,39.0,POINT (-413166.342 4935683.000),30,2,-1,676.0,2022-12-31 23:45:00,2022,12,31,23,45,5,52.0,40.478232,-3.711536
770878,48.0,POINT (-410810.432 4930073.380),5,1,0,680.0,2022-12-31 23:45:00,2022,12,31,23,45,5,52.0,40.439890,-3.690373


1- Check context and add context (smart data model link) to the @context tag

2- Geometry: Change geometry to location. Location is a geoproperty. Give the point values as "coordinates". Keep the observedat on every property.

3- Convert coordinates to lon/lat

4- Add no2, so2 as property (not as a separate entity). One entity per hexagon
. 
5- Add district instead of hexagons


In [9]:


example_flag=False
for property_index, tmp_gdf in enumerate(gdf_list): # for each dataset 
    ingestion_starttime= time.perf_counter() 
    example_flag=True
    sensor_name =gdf_property_list[property_index]
    if(sensor_name == 'PM2.5'):
        sensor_name = "PM2_5"
    counter =0
    uniqueStationIDs= tmp_gdf['station_id'].unique()
    dataset_length= len(tmp_gdf)
    feature_list= tmp_gdf.columns.tolist()

    for u in uniqueStationIDs:
        station_tmp_gdf = tmp_gdf[tmp_gdf['station_id'] == u] # take data coming only from this station ID (this sensor entity)
        entity_obj = Entity(entity_type, entity_id_prefix + sensor_name + ":ID:" + str(int(u)) ) # initialize the entity for the new station
        print(entity_id_prefix + str(sensor_name) + ":" + str(int(u)))
        counter=0 # start pushing data about this entity

        for index, row in station_tmp_gdf.iterrows():

            lat=0
            lon=0
            if counter %num_rows_batch_publish==0: # create a new clean entity everytime a batch is completed/published to avoid duplicate push of same values 
                entity_obj = Entity(entity_type, entity_id_prefix + sensor_name + ":ID:" + str(int(u))) # (re-)initialize the entity
            for f in range(len(feature_list)):
                if feature_list[f] == 'lat': lat = row[f]
                if feature_list[f] == 'lon': lon=row[f]
                if lat !=0 and lon !=0: # both values are obtained for location, add the location attribute 
                    location_value = Point((lat, lon))
                    entity_obj.add_geoprop("location", location_value, observedAt= datetime.strptime(str(row.datetime), '%Y-%m-%d %H:%M:%S')) # MAKE THIS A POINT (SIVA EXAMPLE)
                elif feature_list[f] == 'data':
                    entity_obj.add_prop(sensor_name, str(row[f]) , observedAt= datetime.strptime(str(row.datetime), '%Y-%m-%d %H:%M:%S'))                 
                elif(feature_list[f] not in ['datetime','type']):
                    entity_obj.add_prop(feature_list[f], str(row[f]) , observedAt= datetime.strptime(str(row.datetime), '%Y-%m-%d %H:%M:%S'))                 
            if example_flag:
                print('INFO: Example data from the dataset to be published - as an example on how data looks.')
                print(entity_obj)
                example_flag=False

            if (counter %num_rows_batch_publish==0 or counter == len(station_tmp_gdf)-1) and  counter!=0: # publish the batch data for entity to scorpio as the rows fulfill the number of rows needed for a batch, or the dataset has come to the end
                publish(entity_obj.to_dict()) # publish the entity
            counter+=1 
    print("... published data for dataset: " +  sensor_name)

    ingestion_endtime= time.perf_counter() 
    print('Info: Dataset is published successfully' 
        
          + ' --- It took ' + str(ingestion_endtime -ingestion_starttime) + ' seconds to publish the dataset with length ' + str(dataset_length))


urn:ngsi-ld:AirQualityObserved:Madrid:PM2_5:24
INFO: Example data from the dataset to be published - as an example on how data looks.
{'id': 'urn:ngsi-ld:AirQualityObserved:Madrid:PM2_5:ID:24', 'type': 'AirQualityObserved', '@context': ['https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld'], 'station_id': {'type': 'Property', 'value': '24.0', 'observedAt': '2022-01-01T00:00:00Z'}, 'geometry': {'type': 'Property', 'value': 'POINT (-417152.48156697437 4927070.647350454)', 'observedAt': '2022-01-01T00:00:00Z'}, 'hexagon_id': {'type': 'Property', 'value': '36', 'observedAt': '2022-01-01T00:00:00Z'}, 'hexagon_x': {'type': 'Property', 'value': '3', 'observedAt': '2022-01-01T00:00:00Z'}, 'hexagon_y': {'type': 'Property', 'value': '3', 'observedAt': '2022-01-01T00:00:00Z'}, 'altitude': {'type': 'Property', 'value': '646.0', 'observedAt': '2022-01-01T00:00:00Z'}, 'year': {'type': 'Property', 'value': '2022', 'observedAt': '2022-01-01T00:00:00Z'}, 'month': {'type': 'Property', 'value': '

# Query temporal entity from Scorpio and get it as Pandas

In [10]:

client_obj = Client("localhost", "9090")
pandas_data= client_obj.get_temporal("urn:ngsi-ld:AirQualityObserved:Madrid:Hexagon:0", entityType="AirQualityObserved", pandasSeries=True, attribute="value")
print(pandas_data)
print(type(pandas_data))



KeyError: 'value'

## BACKUP CODE

import time


for entity_index, tmp_gdf in enumerate(gdf_list): # for each dataset 
        
    feature_list= tmp_gdf.columns.tolist()
    for index, row in tmp_gdf.iterrows():
        if index %num_rows_batch_publish==0: # create a new clean entity everytime a batch is completed/published to avoid duplicate push of same values 
            entity_obj = Entity(entity_type_list[entity_index], entity_id_list[entity_index]) # (re-)initialize the entity
        for i in range(len(feature_list)):
            if feature_list[i] =='type': feature_list[i] ='road_type' # quick fix for the error of "type" conflicting with ngsi-ld "type"
            if(feature_list[i] not in ['datetime','type']):
                entity_obj.add_prop(feature_list[i], str(row[i]) , observedAt= datetime.strptime(str(row.datetime), '%Y-%m-%d %H:%M:%S'))                 
        if index == 0:
            print('INFO: Initial row of the dataset to be published - as an example on how data looks.')
            print(entity_obj)
            
        #ingestion_starttime= time.perf_counter()
        temp_objects.append(entity_obj)
        if (index %num_rows_batch_publish==0 or index == len(tmp_gdf)-1) and  index!=0: # publish the batch data for entity to scorpio as the rows fulfill the number of rows needed for a batch, or the dataset has come to the end
            publish(entity_obj.to_dict()) # publish the entity
    
        f(index%10000 ==0):
            print("... published data count: " +  str(index))

        #print('Info: Ingestion took ' + str(ingestion_endtime -ingestion_starttime) + ' seconds for ' +  str(num_rows_batch_publish) + ' objects')

    print('Info: Dataset is published successfully with entity ID: ' + str(entity_id_list[entity_index]))
